In [1]:
!wget http://vision.ime.usp.br/~arturao/cenipa/Cenipa.zip
!apt install unzip
!unzip Cenipa.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install neo4j --quiet

In [3]:
import pandas as pd
import re
import pickle
import geopy.distance

from data_utils.data_models import (
    Manufacturer, Aircraft, 
    AircraftDamage, Occurence, 
    Consequence
)
from data_utils.neo4j_connector import Neo4jConnector

# Data Loading

In [4]:
FOLDER_PREFIX = "Cenipa" #folder containing csvs and such
airplanes = pd.read_csv(f"./{FOLDER_PREFIX}/aeronave.csv", sep=';')
factor = pd.read_csv(f"./{FOLDER_PREFIX}/fator_contribuinte.csv", sep=';')
occurence = pd.read_csv(f"./{FOLDER_PREFIX}/ocorrencia.csv", sep=';')
occurence_type = pd.read_csv(f"./{FOLDER_PREFIX}/ocorrencia_tipo.csv", sep=';')
recomendation = pd.read_csv(f"./{FOLDER_PREFIX}/recomendacao.csv", sep=';')

## Brief Visualization of the Data

In [5]:
airplanes.head()

,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,...,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,39115,PTNQX,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-711A,P28R,PISTÃO,MONOMOTOR,1202,...,BRASIL,BRASIL,AVIÃO,PARTICULAR,BRIGADEIRO ARARIPE MACEDO,CORRENTINA,DECOLAGEM,PRIVADA,SUBSTANCIAL,0
1,39155,PTLVI,***,AVIÃO,BEECH AIRCRAFT,C90,BE9L,TURBOÉLICE,BIMOTOR,4377,...,BRASIL,BRASIL,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,PRIVADA,NENHUM,0
2,39156,PPPTO,***,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500,...,BRASIL,BRASIL,AVIÃO,REGULAR,AFONSO PENA,ADALBERTO MENDES DA SILVA,ARREMETIDA NO AR,REGULAR,LEVE,0
3,39158,PRLGJ,REGULAR,AVIÃO,BOEING COMPANY,757-225,B752,JATO,BIMOTOR,108862,...,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA,REGULAR,NENHUM,0
4,39176,PRMAA,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A320-232,A320,JATO,BIMOTOR,77000,...,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA,REGULAR,NENHUM,0


In [6]:
factor.head()

,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,39115,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,39115,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
2,39115,PERCEPÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
3,39115,PLANEJAMENTO DE VOO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
4,39115,POUCA EXPERIÊNCIA DO PILOTO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL


In [7]:
occurence.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,03/01/2010,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,03/01/2010,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,03/01/2010,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,04/01/2010,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,05/01/2010,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


In [8]:
occurence_type.head()

,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,39115,PANE SECA,COMBUSTÍVEL | PANE SECA,FUEL
1,39155,VAZAMENTO DE COMBUSTÍVEL,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,39156,FOGO EM VOO,FOGO/FUMAÇA (SEM IMPACTO) | FOGO EM VOO,F-NI
3,39158,SUPERAQUECIMENTO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | SUPERAQU...,SCF-PP
4,39176,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP


In [9]:
recomendation.head()

,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,39115,095/2011,2011-07-21,2011-08-01,2015-12-21,Revisar os requisitos de experiência recente p...,NÃO CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,39115,096/2011,2011-07-21,2011-08-01,2011-09-16,Divulgar o presente relatório final aos operad...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,39156,063/2011,2011-06-30,2011-08-01,2011-09-12,Divulgar este Relatório aos operadores de aero...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,39156,064/2011,2011-06-30,2011-08-01,2014-04-16,Acompanhar junto à autoridade primária de cert...,CUMPRIDA DE FORMA ALTERNATIVA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,39235,205/2011,2011-08-29,2011-09-29,2012-11-06,Realizar uma vistoria de segurança de voo na A...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


## Creating Neo4J Database

In [10]:
db = Neo4jConnector("bolt://localhost:7687", "neo4j", "password")

In [11]:
alphabet_regex = re.compile('[^a-zA-Z ]')

### Create Manufacturers

In [12]:
manufacturers_df = (
    airplanes[['aeronave_fabricante', 'aeronave_pais_fabricante']]
      .drop_duplicates(subset='aeronave_fabricante', keep='first')
)
manufacturers_df['aeronave_fabricante'] = manufacturers_df['aeronave_fabricante'].apply(lambda x: alphabet_regex.sub('', x))
for index, row in manufacturers_df.iterrows():
  manufacturer = Manufacturer(
      row.aeronave_fabricante, 
      row.aeronave_pais_fabricante
  )
  db.create_manufacturer(manufacturer)

NEIVA INDUSTRIA AERONAUTICA - BRASIL
BEECH AIRCRAFT - BRASIL
AEROSPATIALE AND ALENIA - BRASIL
BOEING COMPANY - BRASIL
AIRBUS INDUSTRIE - BRASIL
CESSNA AIRCRAFT - BRASIL
EUROCOPTER FRANCE - BRASIL
 - BRASIL
HELIBRAS - BRASIL
AERO BOERO - BRASIL
EMBRAER - BRASIL
ROBINSON HELICOPTER - BRASIL
PIPER AIRCRAFT - BRASIL
ANTONIO CESAR DA SILVA AMARAL - BRASIL
HAWKER BEECHCRAFT - BRASIL
PILATUS - BRASIL
CIRRUS DESIGN - BRASIL
AEROMOT - BRASIL
FOKKER - BRASIL
LEARJET - BRASIL
BELL HELICOPTER - BRASIL
XAVANTE - BRASIL
DASSAULT AVIATION - BRASIL
MCDONNELL DOUGLAS - BRASIL
SIKORSKY AIRCRAFT - BRASIL
NILSON SULZBACH PERES - BRASIL
EDSON PIMENTEL MATHEUS - BRASIL
CIA AERONAUTICA PAULISTA - BRASIL
BRITTENNORMAN - BRASIL
AIR TRACTOR - BRASIL
CHS AMAZONAS IMP E EXP LTDA - BRASIL
AGUSTA - BRASIL
LET AIRCRAFT INDUSTRIES - BRASIL
FABRICACAO PROPRIA - BRASIL
NORTH AMERICAN - BRASIL
INPAER INDUSTRIA PAULISTA E AERONUTICA - BRASIL
CHINCUL SACAIFI - BRASIL
AIRCRAFT INDUSTRIES - BRASIL
RAYTHEON AIRCRAFT - BRASIL

### Create Occurences

In [13]:
occurence_merged = pd.merge(left=occurence, right=occurence_type, on='codigo_ocorrencia1')
occurence_merged['date'] = occurence_merged['ocorrencia_dia'] + ' ' + occurence_merged['ocorrencia_hora']
occurence_merged['date'] = pd.to_datetime(occurence_merged['date'], format='%d/%m/%Y %H:%M:%S')
occurence_merged['ocorrencia_cidade'] = occurence_merged['ocorrencia_cidade'].apply(lambda x: alphabet_regex.sub('', x))

for index, row in occurence_merged.iterrows():
    occur = Occurence(
        row.codigo_ocorrencia,
        row.date,
        row.ocorrencia_uf,
        row.ocorrencia_cidade,
        row.ocorrencia_tipo_categoria
    )
    db.create_occurence(occur)

40211 - 2010-01-03 12:00:00 AT RJ RIO DE JANEIRO
40349 - 2010-01-03 11:05:00 AT PA BELM
40351 - 2010-01-03 03:00:00 AT RJ RIO DE JANEIRO
39527 - 2010-01-04 17:30:00 AT MT LUCAS DO RIO VERDE
40324 - 2010-01-05 19:25:00 AT RS PELOTAS
39807 - 2010-01-06 17:53:00 AT BA SALVADOR
40215 - 2010-01-07 18:40:00 AT AM COARI
39707 - 2010-01-09 12:30:00 AT AM CANUTAMA
39156 - 2010-01-10 23:15:00 AT PR CASCAVEL
39711 - 2010-01-10 20:00:00 AT MG PAR DE MINAS
39789 - 2010-01-10 21:30:00 AT RS SO PEDRO DO SUL
40069 - 2010-01-10 14:50:00 AT SP SO PAULO
40414 - 2010-01-10 03:00:00 AT DF BRASLIA
39507 - 2010-01-11 17:00:00 AT MT PRIMAVERA DO LESTE
40107 - 2010-01-12 14:30:00 AT PA GURUP
39315 - 2010-01-15 21:22:00 AT AM CANUTAMA
39809 - 2010-01-15 15:00:00 AT SP SO PAULO
39810 - 2010-01-15 18:23:00 AT BA SALVADOR
39828 - 2010-01-15 17:45:00 AT SP SANTOS
44575 - 2010-01-17 12:27:00 AT RJ RIO DE JANEIRO
40310 - 2010-01-18 19:32:00 AT AC CRUZEIRO DO SUL
39320 - 2010-01-19 18:30:00 AT PR FAROL
39851 - 2010-01

41649 - 2010-05-12 19:52:00 AT SP SO PAULO
40855 - 2010-05-13 18:58:00 AT AM MANAUS
41573 - 2010-05-13 16:10:00 AT RJ RIO DE JANEIRO
40853 - 2010-05-14 18:25:00 AT SP MARLIA
41574 - 2010-05-14 23:57:00 AT MG JUIZ DE FORA
40933 - 2010-05-15 20:40:00 AT MT CCERES
41485 - 2010-05-15 12:00:00 AT AM MANICOR
41393 - 2010-05-17 11:15:00 AT PR LONDRINA
41409 - 2010-05-17 14:20:00 AT RS BAG
41054 - 2010-05-18 14:55:00 AT PA MONTE ALEGRE
41174 - 2010-05-18 21:50:00 AT AL MACEI
41053 - 2010-05-19 08:10:00 AT PR CASCAVEL
41132 - 2010-05-20 03:00:00 AT SP SO PAULO
41155 - 2010-05-20 19:35:00 AT SP JUNDIA
41155 - 2010-05-20 19:35:00 AT SP JUNDIA
41177 - 2010-05-20 20:05:00 AT GO GOINIA
41179 - 2010-05-20 22:20:00 AT DF BRASLIA
41178 - 2010-05-21 15:40:00 AT SP CAMPINAS
41172 - 2010-05-22 19:02:00 AT PA PARAUAPEBAS
41235 - 2010-05-23 15:35:00 AT DF BRASLIA
41224 - 2010-05-26 15:50:00 AT GO GOINIA
41252 - 2010-05-26 14:10:00 AT MA SO LUS
41374 - 2010-05-26 14:00:00 AT BA CORRENTINA
41274 - 2010-05-27 

42592 - 2010-09-04 13:00:00 AT SP SO PAULO
42738 - 2010-09-04 18:30:00 AT SP SO PAULO
42715 - 2010-09-05 08:10:00 AT PB JOO PESSOA
42718 - 2010-09-08 19:10:00 AT RS VERANPOLIS
42718 - 2010-09-08 19:10:00 AT RS VERANPOLIS
40823 - 2010-09-09 18:50:00 AT PR PALMAS
42717 - 2010-09-09 19:06:00 AT GO GOINIA
43589 - 2010-09-09 03:26:00 AT SC FLORIANPOLIS
42758 - 2010-09-10 14:45:00 AT SP ITANHAM
42760 - 2010-09-10 11:10:00 AT SP PIRACICABA
42760 - 2010-09-10 11:10:00 AT SP PIRACICABA
42714 - 2010-09-11 15:30:00 AT RJ RIO DE JANEIRO
42716 - 2010-09-11 16:30:00 AT PA SANTARM
42868 - 2010-09-11 20:30:00 AT RJ RIO DE JANEIRO
43043 - 2010-09-11 21:13:00 AT PR CAMB
43171 - 2010-09-12 18:30:00 AT MG DIAMANTINA
42757 - 2010-09-13 19:05:00 AT SP SO PAULO
42838 - 2010-09-15 20:30:00 AT PR CURITIBA
42866 - 2010-09-15 13:09:00 AT DF BRASLIA
42919 - 2010-09-15 18:10:00 AT PA ITAITUBA
42921 - 2010-09-15 21:50:00 AT PA SO FLIX DO XINGU
43042 - 2010-09-15 20:00:00 AT RS CARAZINHO
43064 - 2010-09-15 17:54:00 

43648 - 2011-01-04 15:20:00 AT PR TELMACO BORBA
43628 - 2011-01-05 19:40:00 AT BA CAIRU
43631 - 2011-01-05 15:36:00 AT MG CONFINS
43642 - 2011-01-05 22:06:00 AT SP BAURU
43630 - 2011-01-06 11:30:00 AT SP COLINA
43715 - 2011-01-06 19:58:00 AT RJ RIO DE JANEIRO
45329 - 2011-01-06 12:00:00 AT GO CAIAPNIA
43629 - 2011-01-07 15:45:00 AT SP MOGI DAS CRUZES
43637 - 2011-01-09 21:32:00 AT PR SO JOS DOS PINHAIS
43639 - 2011-01-09 18:58:00 AT SP SO PAULO
43652 - 2011-01-09 21:30:00 AT RJ RIO DE JANEIRO
43811 - 2011-01-09 13:30:00 AT MG PATOS DE MINAS
43818 - 2011-01-09 19:15:00 AT PA BELM
43829 - 2011-01-09 18:15:00 AT RJ RIO DE JANEIRO
43649 - 2011-01-10 03:00:00 AT PE NO IDENTIFICADA
43653 - 2011-01-10 14:15:00 AT ES VITRIA
43779 - 2011-01-10 16:35:00 AT SP CAMPINAS
44007 - 2011-01-10 18:18:00 AT RJ CAMPOS DOS GOYTACAZES
43634 - 2011-01-11 09:10:00 AT SP MARLIA
43656 - 2011-01-11 15:45:00 AT RJ MARIC
43657 - 2011-01-11 19:30:00 AT RS DOM PEDRITO
43705 - 2011-01-14 20:10:00 AT GO SENADOR CANEDO

44403 - 2011-04-17 17:15:00 AT TO NOVA ROSALNDIA
44405 - 2011-04-17 18:00:00 AT SP CAJAMAR
44489 - 2011-04-17 16:59:00 AT DF BRASLIA
44395 - 2011-04-18 17:01:00 AT SP RIBEIRO PRETO
44424 - 2011-04-18 20:30:00 AT DF BRASLIA
44437 - 2011-04-18 17:34:00 AT PR LONDRINA
44402 - 2011-04-19 13:35:00 AT SP BAURU
44510 - 2011-04-19 20:00:00 AT GO JATA
44404 - 2011-04-20 11:00:00 AT PA JACAREACANGA
44404 - 2011-04-20 11:00:00 AT PA JACAREACANGA
44420 - 2011-04-20 16:35:00 AT RJ PARATY
44442 - 2011-04-20 15:09:00 AT MG UBERABA
44436 - 2011-04-21 18:51:00 AT AM MANAUS
44677 - 2011-04-22 20:05:00 AT AL MACEI
44492 - 2011-04-23 03:45:00 AT BA PAULO AFONSO
44422 - 2011-04-24 14:17:00 AT RJ RIO DE JANEIRO
44441 - 2011-04-25 11:30:00 AT AM COARI
44480 - 2011-04-25 08:30:00 AT MG BELO HORIZONTE
44434 - 2011-04-26 22:58:00 AT SP SO PAULO
44458 - 2011-04-28 10:55:00 AT MG BELO HORIZONTE
44892 - 2011-04-28 23:49:00 AT AM MANAUS
44481 - 2011-04-29 13:00:00 AT RJ MACA
44893 - 2011-04-29 17:00:00 AT GO GOINIA

...

79196 - 2020-02-17 11:20:00 AT GO INACIOLNDIA
79261 - 2020-02-17 10:44:00 AT SP CAMPINAS
79186 - 2020-02-18 11:40:00 AT GO RIO VERDE
79198 - 2020-02-19 16:15:00 AT MG PATROCNIO
79206 - 2020-02-19 18:50:00 AT ES GUARAPARI
79206 - 2020-02-19 18:50:00 AT ES GUARAPARI
79185 - 2020-02-20 12:24:00 AT AM MANAUS
79195 - 2020-02-20 14:45:00 AT RO PORTO VELHO
79201 - 2020-02-20 20:00:00 AT MT QUERNCIA
79221 - 2020-02-20 15:30:00 AT SP JUNDIA
79183 - 2020-02-22 15:20:00 AT PR CURITIBA
79184 - 2020-02-23 13:00:00 AT MT SO FLIX DO ARAGUAIA
79241 - 2020-02-24 15:25:00 AT MS NOVA ANDRADINA
79199 - 2020-02-25 10:30:00 AT MG PARACATU
79222 - 2020-02-25 16:08:00 AT PR LONDRINA
79223 - 2020-02-25 13:50:00 AT SP GUARUJ
79224 - 2020-02-25 02:10:00 AT SP PIRACICABA
79255 - 2020-02-26 08:30:00 AT AL MACEI
79291 - 2020-02-26 22:06:00 AT BA SALVADOR
79293 - 2020-02-26 15:50:00 AT SP CAMPINAS
79294 - 2020-02-26 21:00:00 AT SC FLORIANPOLIS
79197 - 2020-02-27 11:30:00 AT MT VILA RICA
79204 - 2020-02-28 16:00:00 A

79430 - 2020-07-06 17:40:00 AT RJ RIO DE JANEIRO
79428 - 2020-07-07 10:06:00 AT SP CAMPINAS
79431 - 2020-07-07 13:40:00 AT SP GUARULHOS
79437 - 2020-07-08 21:09:00 AT SP SO PAULO
79437 - 2020-07-08 21:09:00 AT SP SO PAULO
79436 - 2020-07-09 20:30:00 AT SP GUARULHOS
79490 - 2020-07-09 18:30:00 AT PR SO JOO DO CAIU
79438 - 2020-07-10 13:00:00 AT PA CURRALINHO
79440 - 2020-07-10 21:15:00 AT MT SAPEZAL
79439 - 2020-07-11 19:50:00 AT SP PIRACICABA
79441 - 2020-07-12 16:14:00 AT PR CURITIBA
79445 - 2020-07-16 18:30:00 AT RO ALTA FLORESTA DOESTE
79446 - 2020-07-16 13:30:00 AT SP LVARES MACHADO
79447 - 2020-07-19 13:30:00 AT RO NO IDENTIFICADA
79448 - 2020-07-20 16:30:00 AT MS CHAPADO DO SUL
79451 - 2020-07-20 18:00:00 AT SP IBINA
79449 - 2020-07-21 00:00:00 AT MG PATROCNIO
79450 - 2020-07-21 20:00:00 AT AM MANAUS
79453 - 2020-07-21 19:50:00 AT ES VITRIA
79727 - 2020-07-21 19:00:00 AT SP CAMPINAS
79458 - 2020-07-22 10:56:00 AT MG SO JOO DEL REI
79455 - 2020-07-23 17:30:00 AT AM ITAMARATI
79457

79688 - 2020-11-25 15:40:00 AT MG TRS MARIAS
79704 - 2020-11-25 13:07:00 AT RJ RIO DE JANEIRO
79691 - 2020-11-26 20:10:00 AT MG BELO HORIZONTE
79694 - 2020-11-26 10:28:00 AT DF BRASLIA
79692 - 2020-11-28 12:50:00 AT SP CATANDUVA
79703 - 2020-11-28 16:00:00 AT PB JOO PESSOA
79696 - 2020-11-29 10:00:00 AT GO QUIRINPOLIS
79697 - 2020-11-30 18:00:00 AT MT POCON
79699 - 2020-11-30 17:00:00 AT SP GUARULHOS
79750 - 2020-12-01 11:30:00 AT GO GOINIA
79701 - 2020-12-02 16:35:00 AT PR QUATRO BARRAS
79702 - 2020-12-02 16:30:00 AT RJ RIO DE JANEIRO
79739 - 2020-12-03 14:42:00 AT SP SO PAULO
79705 - 2020-12-04 15:30:00 AT SP SOROCABA
79707 - 2020-12-05 12:25:00 AT RJ RIO DE JANEIRO
79712 - 2020-12-06 11:00:00 AT PI BOM JESUS
79712 - 2020-12-06 11:00:00 AT PI BOM JESUS
79713 - 2020-12-06 13:30:00 AT RS SANTA VITRIA DO PALMAR
79711 - 2020-12-08 04:23:00 AT RO PORTO VELHO
79714 - 2020-12-08 14:50:00 AT RJ RIO DE JANEIRO
79717 - 2020-12-08 17:00:00 AT MT NO IDENTIFICADA
79725 - 2020-12-08 18:55:00 AT PR

79974 - 2021-03-10 20:30:00 AT DF BRASLIA
79976 - 2021-03-10 20:30:00 AT MT ARAGUAIANA
79966 - 2021-03-11 12:20:00 AT RO PORTO VELHO
79971 - 2021-03-11 12:30:00 AT MT BARRA DO BUGRES
79978 - 2021-03-12 22:30:00 AT SP CAMPINAS
79968 - 2021-03-13 20:00:00 AT SP IGARAT
79969 - 2021-03-13 12:57:00 AT MG CONQUISTA
80053 - 2021-03-13 23:55:00 AT SP SO PAULO
79972 - 2021-03-14 19:47:00 AT MT ALTA FLORESTA
79973 - 2021-03-14 14:35:00 AT RS ERECHIM
79981 - 2021-03-14 17:05:00 AT PA ALMEIRIM
80041 - 2021-03-14 20:08:00 AT SP GUARULHOS
79975 - 2021-03-15 16:50:00 AT AM MANAUS
79977 - 2021-03-15 10:35:00 AT MT POCON
79979 - 2021-03-16 19:40:00 AT MG BELO HORIZONTE
80058 - 2021-03-17 14:18:00 AT MG PAR DE MINAS
79999 - 2021-03-18 20:00:00 AT RR AMAJARI
79985 - 2021-03-19 02:50:00 AT SP SO PAULO
79982 - 2021-03-20 21:31:00 AT PR CASCAVEL
79983 - 2021-03-20 11:25:00 AT RS SANTO NGELO
79988 - 2021-03-20 19:16:00 AT RJ RIO DE JANEIRO
79989 - 2021-03-22 21:31:00 AT RO JIPARAN
79990 - 2021-03-22 13:30:00

80180 - 2021-07-11 16:35:00 AT RJ RIO DE JANEIRO
80187 - 2021-07-11 20:43:00 AT SP RIBEIRO PRETO
80190 - 2021-07-11 21:30:00 AT PA SANTARM
80176 - 2021-07-12 19:10:00 AT SP BEBEDOURO
80176 - 2021-07-12 19:10:00 AT SP BEBEDOURO
80189 - 2021-07-14 11:50:00 AT MG CONFINS
80222 - 2021-07-14 23:49:00 AT BA MACABAS
80193 - 2021-07-16 14:00:00 AT SP CAMPINAS
80194 - 2021-07-16 07:54:00 AT PR SO JOS DOS PINHAIS
80197 - 2021-07-17 12:15:00 AT SP CAMPINAS
80192 - 2021-07-18 11:58:00 AT SP SO PAULO
80191 - 2021-07-19 13:00:00 AT SP SOROCABA
80195 - 2021-07-21 12:12:00 AT SP JUNDIA
80198 - 2021-07-22 14:45:00 AT GO GOINIA
80204 - 2021-07-22 15:15:00 AT PE RECIFE
80202 - 2021-07-23 11:25:00 AT RJ CAMPOS DOS GOYTACAZES
80205 - 2021-07-23 11:15:00 AT AM MANAUS
80206 - 2021-07-23 22:00:00 AT SP CAMPINAS
80226 - 2021-07-23 18:10:00 AT RS CAXIAS DO SUL
80203 - 2021-07-24 20:10:00 AT SP TUPI PAULISTA
80209 - 2021-07-24 13:50:00 AT GO GOINIA
80199 - 2021-07-25 11:50:00 AT SP ATIBAIA
80200 - 2021-07-25 12:

In [15]:
# import pre-generated mapper from airport to lat long
with open("./airport_geoloc.pkl", "rb") as f:
  mapped_airports = pickle.load(f)

airplanes_geoloc = pd.merge(left=airplanes, right=pd.DataFrame(mapped_airports).T.reset_index(), left_on="aeronave_voo_origem", right_on="index", suffixes=(None, "_departure"))
airplanes_geoloc = pd.merge(left=airplanes_geoloc, right=pd.DataFrame(mapped_airports).T.reset_index(), left_on="aeronave_voo_destino", right_on="index", suffixes=(None, "_arrival")).drop(["index", "index_arrival"], axis=1).dropna()
airplanes_geoloc['coords'] = list(
    zip(
        list(zip(airplanes_geoloc.latitude, airplanes_geoloc.longitude)), 
        list(zip(airplanes_geoloc.latitude_arrival, airplanes_geoloc.longitude_arrival))
    )
)
airplanes_geoloc['distance'] = airplanes_geoloc.coords.apply(
    lambda x: geopy.distance.geodesic(x[0], x[1]).km
)
airplanes_cleaned = pd.merge(left=airplanes, right=airplanes_geoloc[['aeronave_matricula', 'distance']], on='aeronave_matricula', how='left')
airplanes_cleaned['aeronave_fabricante'] = airplanes_cleaned['aeronave_fabricante'].apply(lambda x: alphabet_regex.sub('', x))

for index, row in airplanes_cleaned.iterrows():
    aircraft = Aircraft(
        row.aeronave_matricula,
        row.aeronave_tipo_icao,
        row.aeronave_fabricante,
        row.aeronave_modelo,
        row.aeronave_motor_tipo,
        row.aeronave_motor_quantidade,
        row.aeronave_assentos,
        row.aeronave_ano_fabricacao,
        row.distance
    )
    db.create_aircraft(aircraft)

PTNQX - EMB-711A
PTLVI - C90
PPPTO - ATR-72-212A
PRLGJ - 757-225
PRMAA - A320-232
PTMZU - A320-232
PTWKN - A188B
PTYRE - AS 350 BA
PUFLK - RV-9
PTHLE - HB-350B
PPGMH - AB-115
PPGOB - AB-115
PPGOB - AB-115
PPGOB - AB-115
PTTAF - EMB-110
PRUGO - EMB-810C
PTFNI - A188B
PTOHL - 210N
PTYSG - R44
PUMEL - MISTRAL 582C
PRMAA - A320-232
PPGLS - AB-115
PTNIJ - EMB-711C
PTMFU - ATR-42-300
PTUML - EMB-202
PRSAM - F33A
PTUTM - EMB-202A
PTURT - EMB-202
PRMYE - A320-214
PTUPW - EMB-202
PTEVH - EMB-810C
PTOLF - PA-31T1
PTOLF - PA-31T1
PTZPJ - LANCAIR SUPER E
PTSOG - EMB-110P1
PTEHG - EMB-820C NAVAJO
PRWAR - A188B
PRESP - B200GT
PRLJR - PC-12/47
PPFKI - AB-115
PTGPL - EMB-201
PTBXF - 150E
PTVFM - EMB-712
PTEZN - EMB-820C NAVAJO
PPEOD - AS 350 B2
PRLJS - SR22
PTMXA - A321-231
PRMHZ - A320-214
PRJFS - AMT-600
PROAM - F28MK0100
PTNYV - EMB-712
PTNYV - EMB-712
PTMZH - A320-232
PTMEA - 208B
PTMEA - 208B
PPGMA - AB-115
PTLJK - 35A
PTWKQ - 550
PTOID - 310R
PTHOZ - 206B
PRGEA - 737-7EH
PTNZO - EMB-711ST
PTAFX 

PPGJZ - AB-115
PPGNY - AB-115
PTHOL - R22
PTMEV - 208B
PTREL - EMB-721D
PTWUG - C90A
PTLEU - 95-B55
PTNXS - EMB-712
PTEPY - EMB-820C NAVAJO
PPGOZ - AB-115
PRWJU - 737-36N
PRWJO - 737-3Q8
PTWJW - PA-30
PTVAE - EMB-810D
PPLRR - 60
PTIPO - PA-31
PTMFM - ATR-42-300
PRWJJ - 737-341
PTYYG - R22
PTIUQ - 402B
PRLEH - LA-4-200
PTDZR - PA-28-140
PRGLP - PA-25-235
PUMVB - STOL CH 701
PUMBH - EXCEL CARGO
PTTMD - A319-132
PRGLK - 737-322
PRWJO - 737-3Q8
PRAYE - ERJ 190-200 IGW
PRAYE - ERJ 190-200 IGW
PRMYI - A320-214
PRMBN - A319-132
PRMBY - A320-232
PTXLI - 35A
PRAZC - ERJ 190-100 IGW
PRAZC - ERJ 190-100 IGW
PRAZC - ERJ 190-100 IGW
PROAG - F28MK0100
PTBBB - 31A
PRMHD - A320-214
PRMBA - A320-232
PRCHV - EC 225 LP
PRNOA - L410UVP-E20
PRGLK - 737-322
PTEHG - EMB-820C NAVAJO
PTHVW - R22 BETA
PTXRK - AT-401B
PRGGM - 737-8EH
PRPSK - EMB-145LR
PTMFJ - ATR-42-320
N305FE - DC-10-30F
PRJFC - B300
PPGNI - AB-115
PPKBS - A-122-B
PTRCB - EMB-810C
PRMHZ - A320-214
PTNXJ - EMB-712
PPMAH - S-76A
PROMN - AW139
PTH

PTCTL - V35
PTLJJ - 550
PPFLX - AB-115
PUHHH - P2002 SIERRA RG
PRZTL - RV-10
PRGAB - TBM700N
PTJKV - 210L
PPGRQ - AB-115
PRSEC - S-76C
PRWJG - 737-322
PTMZE - A319-132
PRAZX - ATR-72-202
PROAL - F28MK0100
PTRKL - EMB-711ST
PUPYM - SEAMAX M22
PRTMB - C90A
PRRCH - AS 350 B2
PTYEC - R22
PRJLD - A188B
ZPBCK - C206
PUFTT - SG-ES COYOTE
PRHBC - AS 355 N
PPGAX - AB-115
PTNNW - EMB-710C
PUTJF - D3
PTEMR - EMB-810C
PPKBU - A-122-A
PROPJ - PA-46-350P
PTTNF - EMB-505
PRAYC - ERJ 190-200 IGW
PPRTO - CAP-4
PTLPK - 550
PRCHY - EC 225 LP
PRGUL - 737-8EH
PRMHS - A320-214
PPHOR - 56-C-1
PROAS - F28MK0100
PPMBA - S-76A
PRLCR - S-76C
PPGMR - AB-115
PPGMM - AB-115
PRMEW - BO-105S
PRMBF - A320-232
PTEEK - EMB-810C
PUDVA - VENTURA
PTUJE - EMB-202
PRKLA - R44 II
PTGUV - EMB-201A
CSTOD - A340-312
PRSPJ - EMB-500
PRVMG - 210N
PPGLA - AB-115
PRCVJ - SR20
PTUOS - EMB-202
PTHVL - R22 BETA
PPFLK - AB-115
PPFAX - AB-115
PPGMG - AB-115
PTUCU - EMB-201A
PRLJL - SR20
PTIOT - 210L
PTUOA - EMB-202
PRSER - AT-502B
PPFHP 

PPKBP - A-122-B
PRPNM - EMB-500
PTREI - EMB-721D
PTREI - EMB-721D
PRDGO - PA-31T
PTVHV - EMB-712
PRWJM - 737-36Q
N5527L - 152
PRAYH - ERJ 190-200 IGW
PTRTG - EMB-810D
PRHRU - S-61N
PPAML - 400A
PTRAC - EMB-810C
PRAUL - 58
PRFPM - EC 120 B
PRAVB - A319-115
PRHBF - AS 355 N
PRAZD - ERJ 190-100 IGW
PRAZD - ERJ 190-100 IGW
PTEUW - EMB-810C
PTVGD - EMB-810D
PPMVT - PC-12/47E
PPPTI - ATR-42-320
PRMON - 525B
PTLSJ - 35A
PRIND - 400A
PTHNE - HB-350B
PPMAQ - R22 BETA
PRMPL - R44 II
PTRLJ - EMB-712
PRFSW - D55
PTJBL - 210L
PTAVD - 150J
PTFCS - FALCON 2000EX
PRVBW - 737-7BX
PRASP - DA20-C1
PRNHB - L410UVP-E20
PRAYK - ERJ 190-200 IGW
PRAYK - ERJ 190-200 IGW
PTFTB - 560
PREAJ - 152
PROTK - PC-12
PRCML - PA-32R-301T
PRLGA - A188B
PTLHE - PA-31
PRBRO - 152
PRAPM - AMT-200SO
PTVIR - EMB-810D
PRLTT - PA-34-220T
PTUAD - EMB-201A
PRFAN - EMB-721C
PRMRG - 525B
PUDGM - FOX VECTOR V6
PPBPP - PA-34-220T
PPGRU - AB-115
PRWJD - 737-3Y0
PRWJD - 737-3Y0
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PR

PTGQK - EMB-201A
PTWCM - EMB-110
PRDTX - A188B
PULJR - SEAMAX M22
PPXLR - COMPAIR 8 ANFHIBIAN
PRBGQ - S-76C
PTAVJ - 150J
PPGIX - AB-115
PTEFI - EMB-721C
PTVDJ - EMB-810D
PPGXU - 56-C
PRLON - 58
PUMSN - AA-2120
PTWIN - PA-32R-301
PRZSZ - EXPLORER
PRGGV - 737-8EH
PRHRZ - 212
PUCBR - JABIRU J250
PUTHG - CORSARIO MK5
PRIOB - 727-264
PRGJC - 31A
PRNEC - D55
PTVHV - EMB-712
PTOTC - BAE125-800B
PRLDC - S-76C
PRLCM - S-76C
PRABB - 767-316F
PTTMG - A319-132
PRGUF - 737-8EH
PPALS - EC 155 B1
PTDIS - PA-30
PTRXY - EMB-711ST
PTNIK - EMB-711C
PUMIM - FOX II
PRAPH - PA-32-300
PTMMX - PA-34-200T
PTSUL - AT-502B
PTISH - PA-34-200
PTOQZ - PA-25-235
PUMIM - FOX II
PPGOB - AB-115
PPGOB - AB-115
PPGOB - AB-115
PTORI - M20M
PTVUB - EMB-202A
PRGEA - 737-7EH
PTCQF - 172G
PRGUG - 737-8EH
PRAXJ - ERJ 190-200 IGW
PRAXA - ERJ 190-200 IGW
PRAXA - ERJ 190-200 IGW
PRAXA - ERJ 190-200 IGW
PROMH - EC 155 B1
PROED - R22 BETA
PRAYH - ERJ 190-200 IGW
PRGUU - 737-8EH
PRBVO - 550 BRAVO
PPPLE - 58
PPJGO - 58
PPFPO - EMB-71

...

PRKDT - PA-28-161
PRAUC - ERJ 190-200 IGW
PRLEF - 182N
PTXPC - A321-211
PPGBC - AB-115
PPGBC - AB-115
PPGBC - AB-115
PPGBC - AB-115
PREJK - 152
PRJEA - 150M
PRFFS - 152
PRGGD - 737-8EH
PRGGD - 737-8EH
PPEFM - AS 350 B2
PPEFM - AS 350 B2
PPNEO - 429
PRDOA - 208B
PRZTE - CHRISTEN EAGLE II
PRIPM - 208B
PRAXD - ERJ 190-200 IGW
PTEET - EMB-810C
PPGSZ - 56-C
PTZMI - RV-10
PRFFN - 152
PPCIC - SR22
PUMHV - P2004 BRAVO
PTGMA - 310R
PRONK - A320-214
PRDMC - PA-34-220T
PTRUH - EMB-810D
PTRUH - EMB-810D
PTRUH - EMB-810D
PRBEN - R44 II
PRONY - A320-214
FAB5933 - A29B
PTVLO - EMB-810D
PPECF - AS 350 BA
PTNTH - EMB-712
PPMMR - 210L
PUASP - P92 GOLF
PPSZN - C90GTI
PUOGL - CONQUEST 180
PPGNY - AB-115
PTNAZ - EMB-711C
PRAIZ - A330-243
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PRTTK - ATR-42-500
PTMEJ - 208B
PTMEJ - 208B
PRGTG - 737-8EH
PTBKR - 150J
PPLAV - G58
PRDIB - 45
PROCD - A320-214
PRMYR - A320-214
PTRKA - EMB-711ST
PPGBL - AB-180
PTPPA - L23
PTMXA - A321-231
PTLJ

PRARJ - PA-32R-301
PRZMP - RV-10
PTDNE - 172K
PRRMH - T206H
PTURD - EMB-202
PTMJQ - C90A
PRZER - REGENTE -42 ELO
PTKDU - 182P
PRAGI - AT-502B
PTCON - PA-28-140
PTDVP - PA-28R-200
PRHFA - AS 350 B2
PTOVL - EMB-810C
PTOVL - EMB-810C
PUETD - MC01
PUTKK - ADVENTURE
PTVNH - EMB-720D
PUSUC - SUPER PETREL LS
PRRAU - 58
PRRAU - 58
PTOBL - EMB-810C
PTJDG - 182P
PUSRT - CONQUEST 180
PRMCD - PA-31-350
PREAM - AS 350 B3
PTLSO - C90
PTNKK - EMB-710C
PUCCL - MISTRAL 582C
PTYZF - 407
PPDFM - 120
PRAPN - R44 II
PUMSM - CT-U
PTIUK - 182P
PTUSO - EMB-202
PUVVS - ADVENTURE
PPCHB - 152
PUAFA - FOX IIB
PRZQA - COZY MARK IV
PRMPY - ATR-72-202
PTMHC - 208B
PPZOJ - RV-10
PTUEK - EMB-201A
PRZJO - BEARHAWK
PUXIK - ALTO HENKEL
PUUMG - CONQUEST 180
PTOGH - 208
PRTHI - 152
PRIPM - 208B
PTYTJ - AS 350 B2
PRAUQ - ERJ 190-200 IGW
PRAUQ - ERJ 190-200 IGW
PRAUQ - ERJ 190-200 IGW
PRCJD - AS 350 B2
N100QR - ***
PRCHE - S-76C
PUGAT - FOX V-4
LVGVB - ***
PRKAM - PA-46R-350T
PUPRI - RANS COYOTE
PTRZN - EMB-711ST
PRCHS - S-9

PSAEC - ERJ 190-400
PRAXC - ERJ 190-200 IGW
PRAXC - ERJ 190-200 IGW
PRPSS - SR22
PRTKJ - ATR-72-212A
PRTKJ - ATR-72-212A
PRTKJ - ATR-72-212A
PRTYJ - A320-214
PRAJL - A36
PTCEC - M-4
PRGTO - 737-8EH
PRASE - 150L
PPWFM - R44
PUFMD - RV-9A
PTWSJ - B300
PRDDQ - PA-42-720
PRDDQ - PA-42-720
PRHMJ - S2R-T34
PTMFE - ATR-42-300
PTWNK - T188C
PTVFW - EMB-720D
PTWLN - T188C
PTWXI - EMB-202A
PTUGA - EMB-201A
PTGQI - EMB-201
PRXIG - AT-502B
PRFLE - 152
PRFLE - 152
PRFLE - 152
PRFLE - 152
PTEAQ - EMB-720C
PTAOP - PA-28-180
N36DA - A40A3F
PTGYU - EMB-201A
PTCPX - EMB-203
PTHVQ - R22
PPFLF - AB-115
PROHE - S-92A
PRGGF - 737-8EH
PTHYA - AS 355 F2
PRMHX - A320-214
PTEDF - EMB-820C NAVAJO
PRMAL - A319-132
PTVOZ - EMB-810D
PTKCS - 172M
PPDXR - 170A
PPSAU - AS 350 B2
PRTGG - C90GTI
PRGTP - 737-8EH
PRGTP - 737-8EH
PRVBU - 737-76N
PTGUF - EMB-201A
PUIHS - CONQUEST 180
PRENZ - AT-502B
PRSPI - AS 350 B2
PTPAZ - LK-10A
PRGSG - 310R
PTLZC - PA-25-260
PTKIW - PA-34-200
PTNKE - EMB-711C
PRGUS - 172R
PPMMX - 45
PTO

PPHRH - 56-C-1
PSAEC - ERJ 190-400
PSAEC - ERJ 190-400
PRLKA - A188B
PRLKA - A188B
PRHEB - 206L-4
PRAQJ - ATR-72-212A
PRMHA - A320-214
PPPTP - ATR-72-212A
PPPTP - ATR-72-212A
PPPTP - ATR-72-212A
PPJRL - R22 BETA
PPJRL - R22 BETA
PRTTP - 727-2M7
PRAQK - ATR-72-212A
PRAQK - ATR-72-212A
PRAQK - ATR-72-212A
PPYBB - ***
PPYBB - ***
PROZA - 152
PRVCG - E55
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRAQL - ATR-72-212A
PRAQL - ATR-72-212A
PRYRH - A320-251N
PRAQJ - ATR-72-212A
PRYRF - A320-251N
PPMVC - AT-502B
PTUXD - EMB-202
PRBGM - S-92A
PRNEN - PA-25-235
PRABF - PA-25-235
PPPEG - F33A
PTUON - EMB-202
PRREN - A188B
PTVZA - EMB-202A
PUUAM - SKYPAX
PPMSA - 206B
PPFXH - EMB-711ST
PTJKV - 210L
PTECM - EMB-720C
PRAQO - ATR-72-212A
PRAQO - ATR-72-212A
PRORV - T210M
PRGGF - 737-8EH
PPLCQ - PA-31T
PTDZF - A188A
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRAUJ - ERJ 190-200 IGW
PRGUD - 737-8EH
PRATC - B200
PRATC - B200


PRYSD - ***
PRTTO - 727-2M7
PTWPN - C90A
PRATV - ATR-72-212A
PRATV - ATR-72-212A
PRCMO - 172RG
PRJBB - 152
PTAXH - PA-11
PPZDK - SZD 51 1
PROMB - EC 155 B1
PUSCS - PIX
PRAXF - ERJ 190-200 IGW
PRAXF - ERJ 190-200 IGW
PRAXK - ERJ 190-200 IGW
PRAXK - ERJ 190-200 IGW
PRAUO - ERJ 190-200 IGW
PRAUO - ERJ 190-200 IGW
PRAUO - ERJ 190-200 IGW
PTRUO - EMB-810D
PTWYP - 208B
PRFIB - PA-34-220T
PRVNB - S2R-H80
PTKKF - 210L
PTYNN - AS 350 B2
PTZFN - RV-10
PTIEC - 681
PROSD - A36
PRARI - PA-31T2
PRARI - PA-31T2
PRDLM - EMB-500
PRBBP - 680
PTOJE - PA-31T
PTOJE - PA-31T
PRVBX - 737-7BX
PTRQQ - EMB-810D
PTGYG - EMB-201A
PTBKI - PA-30
PRLDL - R44 II
PRPJN - ERJ 190-400
PTNHH - EMB-711C
PSALJ - ***
PTRUH - EMB-810D
PTRUH - EMB-810D
PTRUH - EMB-810D
PTRMM - R44 II
PPGAS - AB-115
PRVRR - 310G
PRSNV - 550
PRLEN - G58
PRMCM - 172S
PRVOY - 172S
PTMDN - 152
PPPVS - EMB-500
PRXMC - 737-8 MAX
PRIAA - PA-34-200T
PRAJY - 737-48E
PRAJY - 737-48E
PRLBR - R44 II
PTRXE - EMB-721D
PRYRE - A320-251N


In [16]:
airplanes_damage = airplanes.copy()

for index, row in airplanes_damage.iterrows():
    airdamage = AircraftDamage(
        row.codigo_ocorrencia2,
        row.aeronave_matricula,
        row.aeronave_tipo_operacao,
        row.aeronave_fase_operacao,
        row.aeronave_nivel_dano,
        row.aeronave_fatalidades_total
    )
    db.create_aircraft_damage(airdamage)

39115 - PTNQX
39155 - PTLVI
39156 - PPPTO
39158 - PRLGJ
39176 - PRMAA
39178 - PTMZU
39235 - PTWKN
39275 - PTYRE
39295 - PUFLK
39315 - PTHLE
39316 - PPGMH
39317 - PPGOB
39318 - PTTAF
39319 - PRUGO
39320 - PTFNI
39321 - PTOHL
39322 - PTYSG
39323 - PUMEL
39365 - PRMAA
39367 - PPGLS
39385 - PTNIJ
39386 - PTMFU
39405 - PTUML
39487 - PRSAM
39507 - PTUTM
39527 - PTURT
39627 - PRMYE
39687 - PTUPW
39707 - PTEVH
39709 - PTOLF
39711 - PTZPJ
39727 - PTSOG
39747 - PTEHG
39748 - PRWAR
39768 - PRESP
39772 - PRLJR
39787 - PPFKI
39789 - PTGPL
39807 - PTBXF
39809 - PTVFM
39810 - PTEZN
39828 - PPEOD
39847 - PRLJS
39848 - PTMXA
39849 - PRMHZ
39850 - PRJFS
39851 - PROAM
39887 - PTNYV
39910 - PTMZH
39913 - PTMEA
39914 - PPGMA
39916 - PTLJK
39930 - PTWKQ
39933 - PTOID
39967 - PTHOZ
39969 - PRGEA
39972 - PTNZO
39987 - PTAFX
39988 - PTFLF
39992 - PRUUT
39993 - PTBKU
40027 - PPFKV
40069 - PPMAW
40087 - PTAKS
40088 - PTJTM
40089 - PTIXG
40107 - PTEOS
40147 - PTWOO
40148 - PTLCI
40149 - PPGYK
40150 - PTJXO
40168 

43819 - PTCOF
43820 - PTANF
43828 - PRAYF
43829 - PRAZC
43831 - PRMBB
43832 - PTOPR
43836 - PTBBB
43837 - PRAIB
43838 - PTMZX
43842 - PUXOA
43869 - PUFAT
43869 - PURHM
43870 - PRJHM
43871 - PTNYJ
43872 - PTEUT
43873 - PTIHI
43874 - PPGMM
43875 - PPHCP
43876 - PROAM
43885 - PTMXD
43894 - PRCFR
43895 - PTMEI
43899 - PRTTI
43900 - PTPSC
43901 - PPGSY
43903 - PPGSZ
43904 - PRDDM
43906 - PPABX
43908 - PRAAK
43909 - PPHPL
43910 - PPHBK
43911 - PUHHC
43912 - PRSED
43913 - PROAT
43914 - PRWJL
43915 - PTRGY
43921 - PURCJ
43945 - PRMHY
43946 - PTMZZ
43973 - PPGMG
43975 - PRCHI
43977 - PTUXS
43978 - PTOIS
43979 - PRWJJ
43980 - PTLQX
43981 - PRGTU
43983 - PRVBX
43984 - PPMBL
43986 - PRSSK
43988 - PTITI
43989 - PTELY
43992 - PTUXX
43994 - PRABM
43998 - PRLCJ
44001 - PTVAE
44003 - PRELO
44004 - PRPRC
44005 - PTAFX
44007 - PRKNA
44008 - PTOMU
44009 - PPCTA
44011 - PPPJI
44012 - PRGIH
44014 - PRFCR
44017 - PRWGO
44022 - PTBKU
44040 - PPGEE
44043 - PTLHT
44045 - PRMPD
44052 - PRAYK
44055 - PRAYC
44060 

45747 - PTDRR
45748 - PTRTH
45749 - PTLSL
45750 - PPZFE
45751 - PRBTW
45752 - PPHPA
45753 - PPGFC
45754 - PPFLH
45755 - PRDIX
45756 - PTEDC
45757 - PPWCA
45759 - PRCRR
45760 - PPPTL
45768 - PRTKH
45769 - PRTKH
45773 - PRNHB
45774 - PRGNC
45775 - PRDDB
45776 - PTTTL
45777 - PRAZD
45778 - PTLTH
45779 - PTHZG
45780 - N118AV
45781 - PPGEV
45783 - PTNCD
45784 - PTLGW
45785 - PRRUS
45786 - PPPTQ
45789 - PPJGE
45790 - PTEQV
45791 - PRCRA
45792 - PRAVC
45793 - PTVAX
45794 - PRWJA
45795 - PPGOB
45796 - PRTTB
45797 - PPGQY
45798 - PPFLZ
45799 - PTRPT
45800 - PRAJR
45801 - PTMXB
45802 - PRSBM
45803 - PRBIO
45804 - PRIOG
45819 - PPCGO
45820 - PTNUH
45821 - PTYSY
45822 - PTVCE
45829 - PTGVI
45830 - PPFLS
45831 - PPHCP
45833 - PRKNB
45834 - PPFFY
45835 - PPFKV
45841 - PTRBQ
45842 - PRVCI
45843 - PTMZN
45844 - PTMZT
45845 - PRGOD
45849 - PPGLS
45850 - PUCEM
45853 - PRHLS
45854 - PRJMS
45856 - PTWKY
45857 - PTYLN
45859 - PTGMA
45860 - PPPJI
45861 - PRMYP
45862 - PRATR
45864 - PRSEE
45865 - PTTRW
45866

47087 - PPLMM
47095 - PRAYB
47096 - PRAYI
47097 - PRAZB
47098 - PRATP
47099 - PRAYT
47100 - PRRLJ
47102 - PRIOD
47108 - PRSLS
47109 - PRJDV
47110 - PRTPL
47111 - PROTK
47112 - PRXDY
47113 - PRHBM
47114 - PPGMA
47115 - PPFKW
47116 - PRJHR
47117 - PPGGK
47131 - PTZKH
47132 - PTNHB
47133 - PRFSF
47134 - PTZHT
47138 - N209B
47142 - PTVAQ
47143 - PRCPS
47144 - PROAM
47145 - PTVAF
47146 - PRGGT
47153 - PTRSI
47154 - PTTOR
47155 - PPHRF
47156 - PTUCN
47158 - PURHR
47159 - PRRMF
47200 - PPFXH
47201 - PTOCQ
47207 - PTXLI
47208 - PPPTC
47209 - PRAYD
47210 - PTJRS
47211 - PREJI
47213 - PRSEF
47217 - PRAZB
47219 - PTODY
47245 - PRAZU
47245 - PRHOM
47246 - PTLLT
47296 - PTGFF
47297 - PTVVK
47298 - PUADB
47299 - PRAVC
47300 - PRGOM
47302 - PTVKZ
47304 - PRJAN
47305 - PRAYE
47309 - PTTAT
47310 - PRGOF
47311 - PRWHZ
47312 - PTWHU
47313 - PTREY
47314 - PPPJQ
47332 - PTWNW
47333 - PRRCE
47334 - PTKIL
47339 - PTHUW
47351 - PTUTT
47353 - PTRJH
47354 - PRLYD
47422 - PPGJI
47423 - PRVBX
47432 - PRBGI
47433 

50980 - PTUIB
50981 - PTUDL
50983 - PRKSF
50984 - PTUFB
50985 - PTVZI
50991 - PRGPB
50992 - PPSSP
50993 - PPHJA
50994 - PTEMC
50995 - PTUEA
50996 - PTGYC
50998 - PTLJB
50999 - PROAF
51000 - PTOIW
51001 - PTROS
51002 - PPGUJ
51003 - PTISH
51004 - PRVAJ
51005 - PTWUZ
51006 - PTGIH
51007 - PROPJ
51008 - PTXBM
51009 - PRWCO
51010 - PRFPL
51011 - PTHTQ
51012 - PRZOP
51013 - PUSOR
51015 - PTWGV
51016 - PUZEP
51017 - PROVA
51018 - PRZHD
51019 - PRCHI
51020 - PTMZX
51021 - PRFIB
51022 - PTYAR
51023 - PTEPW
51024 - PTMDA
51025 - PPGMR
51026 - PPFHN
51027 - PROAL
51031 - PREDW
51032 - PTONB
51033 - PTWPZ
51052 - PTCVP
51053 - PTGTB
51054 - PRCTM
51055 - PTYDW
51071 - PPGKB
51072 - PPEHY
51073 - PRFAR
51074 - PROMK
51075 - PRVIU
51076 - PPPMT
51077 - PTRQK
51078 - PRGGY
51079 - PRBRU
51080 - PRIMI
51080 - PROHI
51081 - PRSEC
51082 - PRAVJ
51083 - PRMHA
51084 - PRGTL
51086 - PRGTC
51087 - PUHVL
51091 - PRAYS
51092 - PRTKH
51093 - PPWRV
51094 - PRJSR
51095 - PRRJJ
51096 - PTHGB
51097 - PTMTT
51098 

...

60751 - PRBGN
60752 - PTDZA
60753 - PUVLC
60756 - PTWZA
60767 - PRNAS
60768 - PTZPH
60769 - PUWNM
60770 - PPEOX
60783 - PTHZL
60784 - PROGE
60785 - PTVXD
60787 - PRBSI
60790 - PTGII
60792 - PUHNF
60793 - PTGYD
60800 - PPLMM
60802 - PURDC
60805 - PRJAY
60808 - PRACP
60809 - PRSID
60810 - PTIYQ
60811 - CCCPJ
60817 - PTEYU
60819 - PRCON
60836 - PRTTW
60837 - PPSAF
60841 - PRGOP
60844 - PRGDV
60845 - PTUCU
60846 - PRGUE
60846 - PROCB
60847 - YV524T
60849 - PRGGF
60853 - PRDMV
60857 - PRJNW
60859 - PTUQJ
60861 - PUIVN
60862 - PTAHE
60864 - PTEOO
60867 - PRCEF
60869 - PPESC
60870 - PRGTP
60872 - PUSTB
60877 - N186DS
60878 - PRPDC
60881 - PTGYD
60882 - PRZRA
60883 - PTVCC
60892 - PPJBL
60894 - PPJMC
60897 - PRHCT
60903 - PTJSE
60904 - PTEBN
60914 - PRTKJ
60917 - PTXPI
60922 - PPMBN
60924 - PTPAC
60925 - CP2609
60927 - PRGOV
60929 - PRFLN
60930 - PPHPL
60936 - PPGQY
60944 - PTVJC
60947 - PRSRM
60947 - PRWKK
60948 - PPKIK
60949 - PTUKT
60951 - PRJRJ
60954 - PUFLO
60955 - PPZVK
60956 - PPSIX
609

67005 - PPJRC
67008 - PRONW
67009 - PRATE
67010 - PTKZR
67011 - PTOPT
67022 - PRBGT
67028 - PRATU
67034 - PROBI
67038 - PRBAR
67040 - PTLKX
67047 - PRMAP
67048 - PRDRY
67053 - PRUST
67054 - PPXCE
67055 - PRAXF
67058 - PRPRF
67066 - PTIYB
67068 - PPZLG
67075 - PRZDJ
67077 - PRAVD
67078 - PRONR
67086 - PRMHA
67096 - PROSB
67098 - PTWCO
67100 - PTWYD
67101 - N483CJ
67114 - PTRPU
67118 - PRHAC
67119 - PRVBM
67121 - PRBEM
67127 - PRONZ
67129 - PTOEY
67136 - PTRNL
67137 - PUMON
67139 - PPGOB
67140 - PRSAC
67142 - PRAVC
67152 - PTKUK
67155 - PRAZF
67158 - PRAXA
67159 - PTGKO
67163 - N723AN
77200 - PRMLY
77202 - PPWGS
77203 - PRGNL
77205 - PRRMZ
77206 - PREDR
77208 - PRIMN
77209 - PRHAA
77211 - PRBGT
77213 - PRHHG
77230 - PRONU
77239 - PRZTL
77244 - PPHRE
77245 - PRZHD
77247 - PTRMB
77248 - PTOTV
77249 - PPFGR
77250 - PTFJA
77251 - PUANS
77252 - PUMMT
77255 - PRLOR
77256 - PPBRV
77257 - PTCKJ
77258 - PRAYO
77259 - PTOCL
77264 - PTBBN
77265 - PPVIT
77267 - PROTO
77271 - PTFFG
77276 - PRMBG
7727

78316 - PPMIG
78317 - PPZWA
78318 - PRSNC
78320 - PRWGO
78321 - PUCBZ
78322 - PPRCA
78324 - PTUII
78325 - PRPRE
78326 - PPPTP
78330 - PPAYB
78332 - PTNXS
78333 - PTFPS
78334 - PRRMB
78336 - PRONE
78337 - PPOEG
78338 - PRSAC
78339 - PTUZK
78340 - PRGDA
78341 - YV2937
78342 - PPMRT
78343 - PTMMS
78344 - PRJEE
78347 - PPZDS
78348 - PPJFB
78349 - PRAXU
78350 - PTICN
78351 - PRJAW
78352 - PPVSM
78353 - PRBDD
78354 - FGSQM
78356 - PRSFJ
78357 - PTNKH
78359 - PTUUE
78360 - PTONX
78363 - PTFJM
78366 - PTRKE
78367 - PRSDA
78368 - PRKLP
78370 - PTUIQ
78371 - PTWJJ
78372 - PTKIL
78373 - PTXXW
78374 - PTRUK
78375 - PTRDZ
78379 - PPZFO
78380 - PTMUG
78383 - PRAQZ
78384 - PPFBR
78387 - PRTYD
78388 - PTPFG
78390 - PUDVD
78391 - PRRMZ
78394 - PRMPS
78396 - PUBEL
78397 - PPMAX
78405 - PTOEB
78406 - PRONS
78407 - PTUQQ
78408 - PRJBX
78409 - PTRUH
78410 - *****
78411 - PPBGG
78411 - PRYRA
78412 - PTNUU
78416 - PTUEW
78418 - TFFIJ
78426 - PTOXK
78427 - PRJOK
78428 - PPPTM
78429 - PTGYZ
78430 - PTUOL
78432

79127 - PTWOT
79128 - PTUXY
79129 - PTOST
79130 - PPJLS
79131 - PTUMC
79132 - PRVNZ
79134 - PPFGR
79135 - PRAYI
79136 - PRYRB
79137 - PRCDJ
79138 - PTUEW
79140 - PRYRE
79141 - PRAZA
79142 - PRYRS
79143 - PRWOT
79147 - PRMBR
79148 - PRAXH
79149 - PRYRN
79153 - PRATV
79154 - PRSMK
79155 - PTGXP
79156 - PRMPJ
79157 - PRMDT
79158 - PPGEE
79160 - PRFLE
79161 - PRYRH
79162 - PRATV
79164 - PRYYB
79165 - PTXPF
79166 - PRAQH
79167 - PRBLO
79168 - PPYBA
79169 - PSAEC
79170 - PRAXC
79171 - PRPSS
79172 - PRTKJ
79173 - PRTYJ
79175 - PRAJL
79176 - PTCEC
79177 - PRGTO
79178 - PRASE
79179 - PPWFM
79180 - PUFMD
79182 - PTWSJ
79183 - PRDDQ
79184 - PRHMJ
79185 - PTMFE
79186 - PTWNK
79195 - PTVFW
79196 - PTWLN
79197 - PTWXI
79198 - PTUGA
79199 - PTGQI
79201 - PRXIG
79203 - PRFLE
79204 - PTEAQ
79206 - PTAOP
79207 - N36DA
79210 - PTGYU
79211 - PTCPX
79212 - PTHVQ
79213 - PPFLF
79214 - PROHE
79215 - PRGGF
79216 - PTHYA
79217 - PRMHX
79218 - PTEDF
79220 - PRMAL
79221 - PTVOZ
79222 - PTKCS
79223 - PPDXR
79224 

79911 - PRNEX
79912 - PRTYQ
79914 - PTWYT
79916 - PRHSC
79917 - PTJFM
79919 - PRGXI
79920 - PRAKB
79921 - PPHRF
79922 - PPGKM
79924 - PRAAD
79925 - PUBJM
79926 - PRAYY
79927 - PRAUF
79943 - PPEPS
79947 - PTOIW
79948 - PSBVI
79949 - PPBBV
79950 - PRAYG
79951 - PRBVB
79953 - PTVXW
79954 - PRGOW
79955 - PTNPV
79956 - PTDVB
79957 - PRYRU
79958 - PSMIX
79959 - PRYRF
79960 - PRAIY
79961 - PREJR
79962 - PTRSJ
79963 - PPODB
79966 - PRHIB
79968 - PPUGS
79969 - PRECG
79970 - PRTHT
79971 - PTFNX
79972 - PRAXH
79973 - PTPKA
79974 - PTKZR
79975 - PTWRU
79976 - PTNOF
79977 - PTELG
79978 - PSAEC
79979 - PRMYQ
79981 - PTOJE
79982 - PUAVD
79983 - PPGKM
79985 - PRSTD
79988 - PRKKA
79989 - PTJGA
79990 - PTEAC
79991 - PPRQY
79992 - PUMHB
79994 - PRDPM
79996 - PPHQU
79997 - PRTNT
79999 - PUGRB
80001 - PPSES
80002 - PTGPU
80004 - PRGJA
80005 - PRZJW
80006 - PRNMA
80007 - PUVAS
80011 - PTWQY
80012 - PROMQ
80014 - PRODE
80015 - PTOZA
80017 - PTRCC
80018 - PRLDG
80019 - PRSLG
80020 - PRCGJ
80021 - PTVAN
80022 

In [17]:
consequences = recomendation.copy()

for index, row in consequences.iterrows():
    conseq = Consequence(
        row.recomendacao_numero,
        row.codigo_ocorrencia4,
        row.recomendacao_dia_assinatura,
        row.recomendacao_dia_encaminhamento,
        row.recomendacao_dia_feedback,
        row.recomendacao_destinatario
    )
    db.create_consequence(conseq)

095/2011 - 39115
096/2011 - 39115
063/2011 - 39156
064/2011 - 39156
205/2011 - 39235
206/D/11 - 39235
207/2011 - 39235
A-009/CENIPA/2014 – 01 - 39275
A-009/CENIPA/2014 – 02 - 39275
A-009/CENIPA/2014 – 03 - 39275
034/2011 - 39316
035/2011 - 39316
471/2012 - 39318
472/2012 - 39318
473/2012 - 39318
474/2012 - 39318
475/2012 - 39318
067/2011 - 39319
068/2011 - 39319
069/2011 - 39319
035/2010 - 39320
036/2010 - 39320
037/2010 - 39320
038/2010 - 39320
032/2011 - 39321
033/2011 - 39321
093/2011 - 39322
094/2011 - 39322
172/D/11 - 39385
097/2011 - 39405
065/2011 - 39507
066/2011 - 39507
IG-509/CENIPA/2017 - 01 - 39687
088/2011 - 39707
089/2011 - 39707
090/2011 - 39707
091/2011 - 39709
092/2011 - 39709
063/D/10 - 39747
064/D/10 - 39747
065/D/10 - 39747
067/D/10 - 39747
A-557/CENIPA/2015 - 01 - 39887
400/2011 - 39913
401/2011 - 39913
402/2011 - 39913
IG-171/CENIPA/2013 - RSV 001 - 39916
IG-171/CENIPA/2013 - RSV 002 - 39916
IG-170/CENIPA/2013 - RSV 001 - 39930
IG-170/CENIPA/2013 - RSV 002 - 39930

A - 042/CENIPA/2013 - RSV 007 - 43719
A - 042/CENIPA/2013 - RSV 008 - 43719
A - 042/CENIPA/2013 - RSV 009 - 43719
A - 047/CENIPA/2013 - RSV 001 - 43721
A - 047/CENIPA/2013 - RSV 002 - 43721
A - 047/CENIPA/2013 - RSV 003 - 43721
A - 047/CENIPA/2013 - RSV 004 - 43721
335/2012 - 43726
336/2012 - 43726
337/2012 - 43726
338/2012 - 43726
339/2012 - 43726
340/2012 - 43726
275/2012 - 43731
276/2012 - 43731
277/2012 - 43731
080/2011 - 43732
A-522/CENIPA/2014 -  001 - 43774
077/2011 - 43776
078/2011 - 43776
079/2011 - 43776
278/2012 - 43778
279/2012 - 43778
280/2012 - 43778
281/2012 - 43778
282/2012 - 43778
A-039/CENIPA/2013-RSV001 - 43808
A-039/CENIPA/2013-RSV002 - 43808
A-039/CENIPA/2013-RSV003 - 43808
A-039/CENIPA/2013-RSV004 - 43808
A-167/CENIPA/2013 - RSV 001 - 43809
A-167/CENIPA/2013 - RSV 002 - 43809
A-167/CENIPA/2013 - RSV 003 - 43809
014/D/11 - 43899
015/D/11 - 43899
A-012/CENIPA/2013-RSV001 - 43899
A-012/CENIPA/2013-RSV002 - 43899
A-012/CENIPA/2013-RSV003 - 43899
A-012/CENIPA/2013-RSV0

A-506/CENIPA/2016 - 01 - 45273
A-506/CENIPA/2016 - 02 - 45273
A-501/CENIPA/2016 – 01 - 45274
A-501/CENIPA/2016 – 02 - 45274
A-509/CENIPA/2018 - 01 - 45279
A-038/CENIPA/2013-RSV001 - 45280
A-038/CENIPA/2013-RSV002 - 45280
A-046/CENIPA/2014 - RSV 001 - 45284
A-515/CENIPA/2017 - 01 - 45285
A-515/CENIPA/2017 - 02 - 45285
A-539/CENIPA/2014 -  001 - 45291
A-582/CENIPA/2014 - 01 - 45331
A-582/CENIPA/2014 - 02 - 45331
A-582/CENIPA/2014 - 03 - 45331
A-032/CENIPA/2014 - RSV 001 - 45392
A-032/CENIPA/2014 - RSV 002 - 45392
IG-536/CENIPA/2014 - 01 - 45397
IG-536/CENIPA/2014 - 02 - 45397
IG-544/CENIPA/2015 - 01 - 45400
IG-544/CENIPA/2015 - 02 - 45400
IG-544/CENIPA/2015 - 03 - 45400
IG-544/CENIPA/2015 - 04 - 45400
A-509/CENIPA/2021 - 01 - 45408
A - 059/CENIPA/2013 - RSV 001 - 45412
A - 059/CENIPA/2013 - RSV 002 - 45412
A-003/CENIPA/2013-RSV001 - 45487
A-003/CENIPA/2013-RSV002 - 45487
A-003/CENIPA/2013-RSV003 - 45487
A-058/CENIPA/2013-RSV001 - 45490
A-529/CENIPA/2017 - 01 - 45525
A-529/CENIPA/2017 - 0

I - 067/CENIPA/2013 – RSV 011 - 47938
A-089/CENIPA/2013 – 001 - 47966
A-089/CENIPA/2013 – 002 - 47966
A-084/CENIPA/2013 - 001 - 47981
A-084/CENIPA/2013 - 002 - 47981
A-086/CENIPA/2013 - 01 - 47983
A-086/CENIPA/2013 - 02 - 47983
A-086/CENIPA/2013 - 03 - 47983
A-086/CENIPA/2013 - 04 - 47983
A-086/CENIPA/2013 - 05 - 47983
A-087/CENIPA/2013-RSV001 - 47992
A-094/CENIPA/2013-RSV001 - 48053
A-097/CENIPA/2013 - RS 001 - 48438
A-103/CENIPA/2013-01 - 48652
A-103/CENIPA/2013-02 - 48652
IG-100/CENIPA/2013-RSV001 - 48662
A-113/CENIPA/2013 – 01 - 48727
A-113/CENIPA/2013 – 02 - 48727
A-111/CENIPA/2013 - RS 001 - 48791
A-112/CENIPA/2013 - RS 001 - 48795
A-109/CENIPA/2013 – 001 - 48796
A-109/CENIPA/2013 – 002 - 48796
A-108/CENIPA/2013 – 001 - 48797
A-108/CENIPA/2013 – 002 - 48797
IG-129/CENIPA/2013 RS 001 - 49113
IG-129/CENIPA/2013 RS 002 - 49113
IG-129/CENIPA/2013 RS 003 - 49113
IG-129/CENIPA/2013 RS 004 - 49113
A-131/CENIPA/2015 – 01 - 49116
A-133/CENIPA/2013 - 001 - 49119
A-127/CENIPA/2013 – 001 - 4

IG-058/CENIPA/2015 - 01 - 53204
IG-058/CENIPA/2015 - 02 - 53204
IG-058/CENIPA/2015 - 03 - 53204
A-061/CENIPA/2015 - 01 - 53224
A-055/CENIPA/2015 – 01 - 53225
A-055/CENIPA/2015 – 02 - 53225
A-065/CENIPA/2015 - 01 - 53232
A-075/CENIPA/2015 – 01 - 53254
A-073/CENIPA/2015 – 01 - 53257
IG-076/CENIPA/2015 - 01 - 53262
IG-076/CENIPA/2015 - 02 - 53262
IG-076/CENIPA/2015 - 03 - 53262
A-066/CENIPA/2015 - 01 - 53264
A-067/CENIPA/2015 - 01 - 53265
A-067/CENIPA/2015 - 02 - 53265
A-079/CENIPA/2015 - 01 - 53266
A-079/CENIPA/2015 - 02 - 53266
A-079/CENIPA/2015 - 03 - 53266
A-079/CENIPA/2015 - 04 - 53266
A-079/CENIPA/2015 - 05 - 53266
A-079/CENIPA/2015 - 06 - 53266
A-079/CENIPA/2015 - 07 - 53266
A-080/CENIPA/2015 - 01 - 53268
G-077/CENIPA/2015 - 01 - 53270
A-081/CENIPA/2015 - 01 - 53306
IG-084/CENIPA/2015 - 01 - 53308
A-087/CENIPA/2015 – 01 - 53325
IG-085/CENIPA/2015 - 01 - 53326
A-088/CENIPA/2015-01 - 53331
A-088/CENIPA/2015-02 - 53331
IG-091/CENIPA/2015 - 01 - 53332
IG-091/CENIPA/2015 - 02 - 53332
IG

IG-155/CENIPA/2017 - 02 - 77276
IG-155/CENIPA/2017 - 03 - 77276
A-156/CENIPA/2017 - 01 - 77278
A-151/CENIPA/2017 - 01 - 77301
A-150/CENIPA/2017 - 01 - 77304
A-150/CENIPA/2017 - 02 - 77304
A-148/CENIPA/2017 - 01 - 77316
IG-146/CENIPA/2017 - 01 - 77320
IG-146/CENIPA/2017 - 02 - 77320
IG-146/CENIPA/2017 - 03 - 77320
A-140/CENIPA/2017- 01 - 77353
A-140/CENIPA/2017- 02 - 77353
A-139/CENIPA/2017 - 01 - 77354
A-137/CENIPA/2014 - 01 - 77371
A-135/CENIPA/2017 - 01 - 77391
A-135/CENIPA/2017 - 02 - 77391
A-135/CENIPA/2017 - 03 - 77391
A-133/CENIPA/2017 - 01 - 77407
A-133/CENIPA/2017 - 02 - 77407
A-133/CENIPA/2017 - 03 - 77407
A-133/CENIPA/2017 - 04 - 77407
IG-129/CENIPA/2017 - 01 - 77431
IG-129/CENIPA/2017 - 02 - 77431
AP-001/CENIPA/2018-01 - 77438
A-127/CENIPA/2017 - 01 - 77441
A-126/CENIPA/2017 - 01 - 77450
A-126/CENIPA/2017 - 02 - 77450
A-126/CENIPA/2017 - 03 - 77450
IG-169/CENIPA/2017 - 01 - 77454
A-122/CENIPA/2017 - 01 - 77470
A-121/CENIPA/2017- 01 - 77474
A-120/CENIPA/2017 - 01 - 77487
A-12

A-120/CENIPA/2019- 03 - 78773
A-121/CENIPA/2019 - 01 - 78774
A-121/CENIPA/2019 - 02 - 78774
A-121/CENIPA/2019 - 03 - 78774
A-121/CENIPA/2019 - 04 - 78774
A-126/CENIPA/2019 - 01 - 78779
A-139/CENIPA/2019- 01 - 78793
IG-063/CENIPA/2019 - 01 - 78799
IG-063/CENIPA/2019 - 02 - 78799
A-148/CENIPA/2019 - 01 - 78836
A-157/CENIPA/2019 - 01 - 79042
A-157/CENIPA/2019 - 02 - 79042
A-001/CENIPA/2020 - 01 - 79043
A-012/CENIPA/2020 - 01 - 79097
A-015/CENIPA/2020 - 01 - 79111
A-015/CENIPA/2020 - 02 - 79111
A-015/CENIPA/2020 - 03 - 79111
A-015/CENIPA/2020 - 04 - 79111
A-005/CENIPA/2020 - 01 - 79120
A-010/CENIPA/2020 - 01 - 79124
IG-008/CENIPA/2020 - 01 - 79125
IG-008/CENIPA/2020 - 02 - 79125
A-018/CENIPA/2020 - 01 - 79128
A-019/CENIPA/2020 - 01 - 79129
A-019/CENIPA/2020 - 02 - 79129
A-021/CENIPA/2020 - 01 - 79131
A-021/CENIPA/2020 - 02 - 79131
A-006/CENIPA/2020 - 01 - 79179
A-025/CENIPA/2020 - 01 - 79195
A-024/CENIPA/2020 - 01 - 79198
A-024/CENIPA/2020 - 02 - 79198
A-029/CENIPA/2020 - 01 - 79199
A-023/